In [ ]:
# 挑戰畫出台灣互動式地圖分布

import pandas as pd

# 切出景點經緯度
list_838 = pd.read_csv('../data/838-地點清單.csv')

new_basic_info_838 = pd.read_csv('../data/838-new_basic_info.csv')

list_838[['latitude', 'longitude']] = list_838['gmap_coordinates'].str.split(",", expand=True)

list_838 = pd.merge(list_838, new_basic_info_838)

list_838.head()

In [ ]:
import folium
from folium.plugins import FastMarkerCluster

# 台灣的中心座標
taiwan_center = [23.6978, 120.9605]

# 建立地圖
m_cluster = folium.Map(location=taiwan_center, zoom_start=8)

# 只放一次標記，並加上 popup
locations = [
    [row['latitude'], row['longitude'], f"<b>🏠{row['gmap_location']}</b><br>📍{row['address']}<br>📖{row['attraction_type']}<br><a href='{row['href']}' target='_blank'>🔗 {row['viewpoint']}</a>"]
    for _, row in list_838.iterrows()
]

# 使用 FastMarkerCluster 並加上 popup
FastMarkerCluster(
    data=[[lat, lon, popup] for lat, lon, popup in locations],
    callback="""
    function (row) {
        var marker = L.marker(new L.LatLng(row[0], row[1]));
        marker.bindPopup(row[2]);
        return marker;
    }
    """
).add_to(m_cluster)

# 儲存地圖為 HTML 檔案
m_cluster.save("../data/taiwan_markers_cluster.html")

# 在 Jupyter Notebook 中顯示
m_cluster